In [ ]:
import dspy

## Introduction

### dspy.Assert and dspy.Suggest API

* dspy.Assert:

    * Parameters:
        * constraint (bool): Outcome of Python-defined boolean validation check.
        * msg (Optional[str]): User-defined error message providing feedback or correction guidance.
        * backtrack (Optional[module]): Specifies target module for retry attempts upon constraint failure. The default backtracking module is the last module before the assertion.
    * Behavior: Initiates retry upon failure, dynamically adjusting the pipeline's execution. If failures persist, it halts execution and raises a dspy.AssertionError.

* dspy.Suggest:

        * Parameters: Similar to dspy.Assert.
        * Behavior: Encourages self-refinement through retries without enforcing hard stops. Logs failures after maximum backtracking attempts and continues execution.
* dspy.Assert vs. Python Assertions: Unlike conventional Python assert statements that terminate the program upon failure, dspy.Assert conducts a sophisticated retry mechanism, allowing the pipeline to adjust.

Specifically, when a constraint is not met:

* Backtracking Mechanism: An under-the-hood backtracking is initiated, offering the model a chance to self-refine and proceed, which is done through
Dynamic Signature Modification: internally modifying your DSPy program’s Signature by adding the following fields:
* Past Output: your model's past output that did not pass the validation_fn
Instruction: your user-defined feedback message on what went wrong and what possibly to fix
If the error continues past the max_backtracking_attempts, then dspy.Assert will halt the pipeline execution, altering you with an dspy.AssertionError. This ensures your program doesn't continue executing with “bad” LM behavior and immediately highlights sample failure outputs for user assessment.

dspy.Suggest vs. dspy.Assert: dspy.Suggest on the other hand offers a softer approach. It maintains the same retry backtracking as dspy.Assert but instead serves as a gentle nudger. If the model outputs cannot pass the model constraints after the max_backtracking_attempts, dspy.Suggest will log the persistent failure and continue execution of the program on the rest of the data. This ensures the LM pipeline works in a "best-effort" manner without halting execution.

dspy.Suggest are best utilized as "helpers" during the evaluation phase, offering guidance and potential corrections without halting the pipeline.

dspy.Assert are recommended during the development stage as "checkers" to ensure the LM behaves as expected, providing a robust mechanism for identifying and addressing errors early in the development cycle.

## Use Case: Including Assertions in DSPy Programs

In [ ]:
class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=2, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []
        prev_queries = [question]

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            prev_queries.append(query)
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)
        
        pred = self.generate_answer(context=context, question=question)
        pred = dspy.Prediction(context=context, answer=pred.answer)
        return pred

baleen = SimplifiedBaleen()

baleen(question = "Which award did Gary Zukav's first book receive?")

In [ ]:
#simplistic boolean check for query length
len(query) <= 100

#Python function for validating distinct queries
def validate_query_distinction_local(previous_queries, query):
    """check if query is distinct from previous queries"""
    if previous_queries == []:
        return True
    if dspy.evaluate.answer_exact_match_str(query, previous_queries, frac=0.8):
        return False
    return True

In [ ]:
dspy.Suggest(
    len(query) <= 100,
    "Query should be short and less than 100 characters",
)

dspy.Suggest(
    validate_query_distinction_local(prev_queries, query),
    "Query should be distinct from: "
    + "; ".join(f"{i+1}) {q}" for i, q in enumerate(prev_queries)),
)

In [ ]:
class SimplifiedBaleenAssertions(dspy.Module):
    def __init__(self, passages_per_hop=2, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []
        prev_queries = [question]

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query

            dspy.Suggest(
                len(query) <= 100,
                "Query should be short and less than 100 characters",
            )

            dspy.Suggest(
                validate_query_distinction_local(prev_queries, query),
                "Query should be distinct from: "
                + "; ".join(f"{i+1}) {q}" for i, q in enumerate(prev_queries)),
            )

            prev_queries.append(query)
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)
        
        if all_queries_distinct(prev_queries):
            self.passed_suggestions += 1

        pred = self.generate_answer(context=context, question=question)
        pred = dspy.Prediction(context=context, answer=pred.answer)
        return pred